# Run eQTL Analysis

This notebook coordinates and executes the eQTL analysis. This notebook is
specialized for the Frazer lab cluster. Since running the entire analysis is 
time consuming, I generally run it "by hand," starting jobs for groups of
genes at different times.

In [1]:
import cPickle
import datetime
import glob
import gzip
import os
import random
import shutil
import subprocess
import time
import uuid

import cdpybio as cpb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools as pbt
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm

import cardipspy as cpy
import ciepy

%matplotlib inline
%load_ext rpy2.ipython

random.seed(20150605)

/frazer01/home/cdeboever/software/anaconda/envs/cie/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
outdir = os.path.join(ciepy.root, 'output',
                      'run_eqtl_analysis')
cpy.makedir(outdir)

private_outdir = os.path.join(ciepy.root, 'private_output',
                              'run_eqtl_analysis')
cpy.makedir(private_outdir)

In [3]:
gene_info = pd.read_table(cpy.gencode_gene_info, index_col=0)

fn = os.path.join(ciepy.root, 'output', 'eqtl_input', 'gene_to_regions.p')
gene_to_regions = cPickle.load(open(fn, 'rb'))

exp = pd.read_table(os.path.join(ciepy.root, 'output', 'eqtl_input', 
                                 'tpm_log_filtered_phe_std_norm_peer_resid.tsv'), index_col=0)

In [4]:
def run_emmax(gene_id):
    os.chdir('/raid3/projects/CARDIPS/analysis/cardips-ipsc-eqtl/notebooks')
    toutdir = os.path.join(outdir, 'results', gene_id)
    if not os.path.exists(toutdir):
        cpy.makedir(toutdir)
        fn = os.path.join(toutdir, '{}.sh'.format(gene_id))
        with open(fn, 'w') as f:
            c = 'python {} \\\n\t'.format(os.path.join(ciepy.root, 'scripts', 'run_emmax.py'))
            c += ' \\\n\t'.join([
                    gene_id,
                    os.path.join(ciepy.root, 'private_data', 'wgs', 'biallelic_snvs.vcf.gz'),
                    ','.join(gene_to_regions[gene_id]),
                    os.path.join(ciepy.root, 'output', 'eqtl_input', 
                                 'tpm_log_filtered_phe_std_norm_peer_resid.tsv'),
                    os.path.join(ciepy.root, 'output', 'eqtl_input', 'emmax.ind'),
                    os.path.join(ciepy.root, 'output', 'eqtl_input', 'wgs.kin'),
                    toutdir,
                    '-c {}'.format(os.path.join(ciepy.root, 'output', 'eqtl_input', 
                                                'emmax_sex_only.cov')),
                ])
            f.write(c + '\n')
        subprocess.check_call('bash {}'.format(fn), shell=True)

## SGE

Run jobs using SGE queue system.

In [241]:
def run_emmax_sge(gene_ids, n=10):
    """
    gene_ids is a list of gene_ids and n is the number of genes to submit at the same time.
    This script will find n number of genes that EMMAX hasn't been run for and submit a job
    for those genes.
    """
    genes_todo = []
    i = 0
    while len(genes_todo) < n:
        if not os.path.exists(os.path.join(outdir, 'results', gene_ids[i])):
            genes_todo.append(gene_ids[i])
        i += 1
    res = datetime.datetime.now()
    #date = '{}-{:02d}-{:02d}-{:02d}-{:02d}-{:02d}'.format(res.year, res.month,
    #                                                      res.day, res.hour,
    #                                                      res.minute,
    #                                                      res.second)
    date = re.sub(r'\D', '_', str(res))
    fn = os.path.join(outdir, 'results', 'sge_scripts', '{}.sh'.format(date))
    with open(fn, 'w') as f:
        f.write('#!/bin/bash\n\n')
        f.write('#$ -N emmax_{}\n'.format(date))
        f.write('#$ -l h_vmem=8G\n')
        f.write('#$ -pe smp 1\n')
        f.write('#$ -S /bin/bash\n')
        f.write('#$ -o {}/emmax_{}.out\n'.format(
                os.path.join(outdir, 'results', 'sge_scripts'), date))
        f.write('#$ -e {}/emmax_{}.err\n\n'.format(
                    os.path.join(outdir, 'results', 'sge_scripts'), date))
        f.write('module load cardips\n')
        f.write('source activate cie\n\n')
        for gene_id in genes_todo:
            toutdir = os.path.join(outdir, 'results', gene_id)
            cpy.makedir(toutdir)
            c = 'python {} \\\n\t'.format(os.path.join(ciepy.root, 'scripts', 'run_emmax.py'))
            c += ' \\\n\t'.join([
                    gene_id,
                    os.path.join(ciepy.root, 'private_output/eqtl_input/filtered_all/0000.vcf.gz'),
                    ','.join([x[3:] for x in gene_to_regions[gene_id]]),
                    os.path.join(ciepy.root, 'output', 'eqtl_input', 
                                 'tpm_log_filtered_phe_std_norm_peer_resid.tsv'),
                    os.path.join(ciepy.root, 'output', 'eqtl_input', 'emmax.ind'),
                    os.path.join(ciepy.root, 'output', 'eqtl_input', 'wgs.kin'),
                    toutdir,
                    '-c {}'.format(os.path.join(ciepy.root, 'output', 'eqtl_input', 
                                                'emmax_sex_only.cov')),
                ])
            f.write(c + '\n\n')
    subprocess.check_call('qsub {}'.format(fn), shell=True)

In [194]:
cpy.makedir(os.path.join(outdir, 'results', 'sge_scripts'))

In [246]:
todo = list(set(exp.index) - 
            set([os.path.split(x)[1] for x in glob.glob(os.path.join(outdir, 'results', '*'))]))
todo = [x for x in todo if gene_info.ix[x, 'chrom'] not in ['chrX', 'chrY', 'chrM']]

In [247]:
# Remove failed genes. I'll wait to resubmit these with more memory.
if os.path.exists(os.path.join(outdir, 'failed.txt')):
    with open(os.path.join(outdir, 'failed.txt')) as f:
        failed = [x.strip() for x in f.readlines()]
todo = list(set(todo) - set(failed))

In [196]:
if len(todo) > 0:
    for i in range(10):
        for j in range(20):
            run_emmax_sge(todo, n=5)
        time.sleep(15)

### Look for failed jobs

Some genes have errors when they are running. This can happen if I don't request enough memory 
for instance. If a job fails on a given gene, any of the genes after that one in that job also
aren't analyzed. These genes won't have `minimum_pvalues.tsv` files even when their job finishes. 
I want to identify these genes and remove their output
directory so they will be run in a new job. I think these genes often leave behind their
temp directory so I'll have to go delete those.

In [264]:
# Get jobs currently waiting to start or started.
running = !qstat -r | grep jobname
running = [x.split()[-1] for x in running if 'emmax_' in x]
# Get all submission scripts created.
fns = glob.glob(os.path.join(outdir, 'results', 'sge_scripts', '*.sh'))
jobnames = []
genes = []
for fn in fns:
    with open(fn) as f:
        lines = [x.strip() for x in f.readlines()]
    genes.append([x.split()[0] for x in lines if x[0:4] == 'ENSG'])
    jobnames.append(lines[2].split()[-1])
jobs = pd.DataFrame(np.array(fns).T, index=jobnames, columns=['path'])
jobs = pd.DataFrame([fns, genes], columns=jobnames, index=['path', 'genes']).T
jobs['status'] = 'finished'
# For now, running means either the job is waiting to start or has started.
jobs.ix[running, 'status'] = 'running'

genes = [os.path.split(x)[1] for x in glob.glob(os.path.join(outdir, 'results', 'ENSG*'))]
genes = pd.DataFrame(index=genes)
genes['status'] = 'incomplete'
min_pvals = glob.glob(os.path.join(outdir, 'results', 'ENSG*', 'minimum_pvalues.tsv'))
genes.ix[[x.split('/')[-2] for x in min_pvals], 'status'] = 'complete'
genes['job_status'] = 'finished'
# If there is any running job with this gene, we want to mark the job_status
# as running.
running_genes = [item for sublist in jobs.ix[jobs.status == 'running', 'genes'].values 
                 for item in sublist]
genes.ix[running_genes, 'job_status'] = 'running'

pd.crosstab(genes.status, genes.job_status)

job_status,finished,running
status,,
complete,1181,348
incomplete,11,897


Any genes that incomplete but whose jobs are finished had errors. I need 
to delete their temp directories and the output directories and try resubmitting.
I may want to resubmit with more memory. I'll keep track of the failed genes so
I can wait to resubmit them with more memory.

In [265]:
for g in genes[(genes.status == 'incomplete') & (genes.job_status == 'finished')].index:
    # Delete temp directory if it exists.
    dy = '/dev/shm/{}'.format(g)
    c = 'pdsh -g n "if [ -d "{0}" ]; then rm -r {0} ; fi"'.format(dy)
    subprocess.check_call(c, shell=True)

    # Remove output directory. 
    dy = os.path.join(outdir, 'results', g)
    c = 'rm -r {}'.format(dy)
    subprocess.check_call(c, shell=True)
with open(os.path.join(outdir, 'failed.txt'), 'a') as f:
    f.write('\n'.join(genes[(genes.status == 'incomplete') & (genes.job_status == 'finished')].index) + '\n')

In [266]:
gene_to_regions['ENSG00000163933.5']

['chr3:52154037-54164478']

In [ ]:
3 +

## Memory requirement

I have to specify the amount of memory needed for each job. I think I can get
away with one thread but the amount of memory scales with the number of variants
(I'm guessing) so I need to figure out the relationship between number of variants
and the amount of RAM needed.